# Programming Assignment 3 - Logistic regression without libraries

CAP 5625 Computational Foundations of AI - Fall 2021 - Dr. DeGiorgio
<br>Christian Garbin

Instructions to run the code are available in the README.md file.

# Setting up the environment

Import the Python modules we use in the notebook and configure the Jupyter environment.

In [1]:
import copy
from typing import Tuple
import numpy as np
import pandas as pd

# Import and set up the graphing environment
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Make plots legible and focus on trends (not specific values)
sns.set_style('white')
sns.set_context('notebook', font_scale=1.5, rc={"lines.linewidth": 2})
sns.despine();

<Figure size 432x288 with 0 Axes>

Import our logistic regression module and the utilities.

In [2]:
import utils
import logistic

# Ancillary code and definitions

Run the automated tests to verify the code works as intended before we start the experiments.

In [3]:
import sys
sys.path.append('./test')
import test_all
# Set to true to see each test result
test_all.test_all(False, './data')
print('All tests passed')

All tests passed


Constants used in the experiments.

In [4]:
LAMBDAS_TO_TEST = [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4]
NUM_TESTS = len(LAMBDAS_TO_TEST)
LR = 0.00001
NUM_FOLDS = 5

Read the dataset and prepare it by encoding categorical columns, standardizing if asked to do so. Note that there is no caching. For a larger dataset we should probably cache it.

Also set constants that depend on the dataset.

In [11]:
FEATURE_NAMES = []
CATEGORIES = []

def read_dataset(standardize: bool = True) -> Tuple[np.ndarray, np.ndarray]:
    global FEATURE_NAMES, CATEGORIES
    
    x, y, FEATURE_NAMES, CATEGORIES = utils.read_dataset(
        './data/TrainingData_N183_p10.csv', hot_encode=True)

    if standardize:
        utils.scale(x)

    return x, y

In [12]:
x, y = read_dataset(standardize=False)

In [16]:
dataset = pd.read_csv('./data/TrainingData_N183_p10.csv')

# Split the features (input) from the output, assumming the last column is the output
features = dataset.iloc[:, :-1]
output = dataset.iloc[:, -1:]

# If requested, convert the output variable from categorical to indicators (hot encoding)
# Note that it assumes that the output is a single column
output = output.iloc[:, 0].str.get_dummies()

# Convert to NumPy arrays in preparation to manipulate it
x = features.to_numpy()
y = output.to_numpy()


In [24]:
CATEGORIES

['African', 'EastAsian', 'European', 'NativeAmerican', 'Oceanian']

# Deliverable 1 - effect of lambda on coefficients

> _Illustrate the effect of the tuning parameter on the inferred ridge regression coefficients by generating five plots (one for each of the K = 5 ancestry classes) of 10 lines (one for each of the p = 10 features), with the y-axis as β̂jk, j = 1,2, ... ,10 for the graph of class k, and x-axis the corresponding log-scaled tuning parameter value log10(λ) that generated the particular β̂jk._

Perform logistic regression with the different lambda values (takes several seconds to complete).

In [ ]:
betas = []
for lmbda in LAMBDAS_TO_TEST:
    x, y = read_dataset()
    b = logistic.fit(x, y, lr=LR, lmbda=lmbda, iterations=100_000)
    betas.append(b)

Plot the results, showing how larger values of lambda decrease the coefficients (betas).

In [ ]:
def plot_betas(betas: np.ndarray, title: str):
    # Create a DataFrame in the long format
    df = pd.DataFrame(np.squeeze(betas), columns=FEATURE_NAMES,
                      index=LAMBDAS_TO_TEST)
    df = df.stack().reset_index()
    df.columns = ['Lambda', 'Feature', 'Standardized Coefficients']
    
    # Plot it
    fig, ax = plt.subplots(figsize=(12, 8));
    sns.lineplot(ax=ax, y='Standardized Coefficients', x='Lambda',
                 hue='Feature', data=df)
    ax.set_xscale('log')
    plt.legend(bbox_to_anchor=(1.04,1), loc='upper left')
    ax.set_title(title)
    
plot_betas(betas, 'Coefficients x lambda')

# Deliverable 2 - effect of lambda on MSE (with cross validation)

> _Illustrate the effect of the tuning parameter on the cross validation error by generating a plot with the y-axis as CV(5) error, and the x-axis the corresponding log-scaled tuning parameter value log10(λ) that generated the particular CV(5) error._

Perform logistic regression with the different lambda values and cross validation (takes several seconds to complete).

In [ ]:
mse = np.zeros(NUM_TESTS)
for i, lmbda in enumerate(LAMBDAS_TO_TEST):
    x, y = read_dataset(standardize=False)
    fold_mse = np.zeros(NUM_FOLDS)
    for fold in range(1, NUM_FOLDS+1, 1):
        x_train, x_val, y_train, y_val = utils.split_fold(x, y,
                                                          num_folds=NUM_FOLDS,
                                                          fold=fold)
        utils.scale_val(x_train, x_val)
        utils.center_val(y_train, y_val)

        model = logistic.fit(x_train, y_train, lr=LR, lmbda=lmbda, iterations=20_000)
        predictions = logistic.predict(x_val, model)
        fold_mse[fold-1] = utils.mse(y_val, predictions)
    mse[i] = fold_mse.mean()

Plot the results.

In [ ]:
def plot_mse(mse: np.ndarray, title: str):
    # Create a DataFrame in the long format to plot
    df2 = pd.DataFrame([LAMBDAS_TO_TEST, mse]).T
    df2.columns = ['Lambda', 'MSE']
    
    # Plot it
    fig, ax = plt.subplots(figsize=(12, 8));
    sns.lineplot(ax=ax, y='MSE', x='Lambda', data=df2)
    ax.set_xscale('log')
    ax.set_title(title)

plot_mse(mse, 'Effect of lambda on MSE')

# Deliverable 3 - the lambda with the smallest MSE

> _Indicate the value of λ value (sic) that generated the smallest CV(5) error._

In [ ]:
smallest_mse_index = mse.argmin()
smallest_mse_lambda = LAMBDAS_TO_TEST[smallest_mse_index]
print(f'The lambda with the smallest MSE is {smallest_mse_lambda}')

# Deliverable 4 - model parameters for the lambda with the smallest MSE

> _Given the optimal λ, retrain your model on the entire dataset of N = 183 observations to obtain an estimate of the (p + 1) × K model parameter matrix as B̂ and make predictions of the probability for each of the K = 5 classes for the 111 test individuals located in `TestData_N111_p10.csv`._

Train a model with the lambda that resulted in the smallest MSE.

In [ ]:
x, y = read_dataset()
betas4 = logistic.fit(x, y, lr=LR, lmbda=smallest_mse_lambda, iterations=100_000)

Show the coefficients for this model.

In [ ]:
for feature, beta in zip(FEATURE_NAMES, betas4.flatten()):
    print(f'{feature:>10}: {beta:7.2f}')

# Deliverable 5 - compare with expected real-world results

> How do the class label probabilities differ for the Mexican and African American samples when compared to the class label probabilities for the unknown samples? Are these class probabilities telling us something about recent history? Explain why these class probabilities are reasonable with respect to knowledge of recent history.

# Deliverable 7 - repeat with a machine learning library

> _Implement the assignment using statistical or machine learning libraries in a language of your choice. Compare the results with those obtained above, and provide a discussion as to why you believe your results are different if you found them to be different._

In this section we repeat the experiments above using scikit-learn. It uses the `sag` solver because it is the closest to the one implemented in our code (a gradient descent algorithm).

In [ ]:
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import pipeline
from sklearn import metrics

## Deliverable 7.1 - effect of lambda on coefficients

In [ ]:
betas_sk = []
for lmbda in LAMBDAS_TO_TEST:
    x, y = read_dataset(standardize=False)
    model_sk = pipeline.make_pipeline(preprocessing.StandardScaler(),
                                      linear_model.Ridge(alpha=lmbda,
                                                         solver='sag'))
    model_sk.fit(x, y)
    coef_sk = model_sk.named_steps['ridge'].coef_
    betas_sk.append(coef_sk)

Plots the results.

In [ ]:
plot_betas(betas_sk, 'Coefficients x lambda (scikit-learn)')

## Deliverable 7.2 - effect of lambda on MSE (with cross validation)

Fit one model for each value of lambda. Ask scikit-learn to store the cross-validation results (`store_cv_values`) so we can calculate the mean MSE for each lambda.

In [ ]:
x, y = read_dataset(standardize=False)
p = pipeline.make_pipeline(preprocessing.StandardScaler(),
                           linear_model.RidgeCV(alphas=LAMBDAS_TO_TEST,
                           store_cv_values=True))
p.fit(x, y)
model_sk = p.named_steps['ridgecv']

# Mean MSE for each fold
mse_sk = model_sk.cv_values_.mean(axis=0)

Plot the results.

In [ ]:
plot_mse(mse_sk.flatten(), 'Effect of lambda on MSE (scikit-learn)')

## Deliverable 7.3 - the lambda with the smallest MSE

In [ ]:
print(f'The lambda with the smallest MSE is {model_sk.alpha_}')

## Deliverable 7.4 - model parameters for the lambda with the smallest MSE

In [ ]:
x, y = read_dataset(standardize=False)
p = pipeline.make_pipeline(preprocessing.StandardScaler(),
                           linear_model.Ridge(alpha=model_sk.alpha_))
p.fit(x, y)
coef_sk = p.named_steps['ridge'].coef_
for feature, beta in zip(FEATURE_NAMES, coef_sk.flatten()):
    print(f'{feature:>10}: {beta:7.2f}')

## Deliverable 7.5 - compare with expected real-world results

## Discussion of differences

TODO